### Market Basket Analysis

The following is one of the assignments for the Data Mining class in the 6th semester. I was assigned to apply market basket analysis to an online retail data.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# Import Dataset
retail = pd.read_csv('online_retail.csv')
retail.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 08:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 08:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 08:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/10 08:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 08:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 08:34,1.69,13047.0,United Kingdom


In [3]:
retail.shape

(541909, 8)

### Detecting Missing Value

In [4]:
np.sum(retail.isnull())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

It is known that there is missing value in the "Description" and "CustomerID" columns.

The "Description" column shows what products are recorded in a transaction. If there is missing value in that column, it is not known what product was recorded. Therefore, observations with missing values in the "Description" column will be deleted / dropped.

The "CustomerID" column shows the identity of the customer involved in the transaction. This column has no effect in the analysis so that any missing values in the column will be ignored.

In [5]:
# Delete Observation with Missing Value in the "Description" column
retail = retail.dropna(subset = ["Description"])
np.sum(retail.isnull())

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64

### Pre-Processing

In [6]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    540455 non-null  object 
 1   StockCode    540455 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     540455 non-null  int64  
 4   InvoiceDate  540455 non-null  object 
 5   UnitPrice    540455 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      540455 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 37.1+ MB


Column "InvoiceNo" contains data with "C" as the prefix. Therefore, these data need to be deleted / dropped

In [7]:
# Delete Data with "C" as the Prefix on "InvoiceNo" Column
retail['InvoiceNo'] = retail['InvoiceNo'].astype('str')
retail = retail[~retail['InvoiceNo'].str.contains('C')]
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531167 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    531167 non-null  object 
 1   StockCode    531167 non-null  object 
 2   Description  531167 non-null  object 
 3   Quantity     531167 non-null  int64  
 4   InvoiceDate  531167 non-null  object 
 5   UnitPrice    531167 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      531167 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.5+ MB


Column "Quantity" contains data with a negative value which indicates a transaction cancellation. Data with this value also needs to be deleted / dropped

In [8]:
# Delete Data with Negative Values on "Quantity" Column
retail = retail[~retail['Quantity'] < 0]
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530693 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    530693 non-null  object 
 1   StockCode    530693 non-null  object 
 2   Description  530693 non-null  object 
 3   Quantity     530693 non-null  int64  
 4   InvoiceDate  530693 non-null  object 
 5   UnitPrice    530693 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      530693 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.4+ MB


### Set Transaction Data

In [9]:
retail["Country"].value_counts()

United Kingdom          485694
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

In [10]:
basket = (retail[retail['Country'] == "Germany"].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket.head()

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
basket.shape

(457, 1697)

It is known that in Germany, 457 transactions were recorded involving 1.697 types of products

### Market Basket Analysis using Association Rule

In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
 
# Delete Postage Column
basket_sets.drop('POSTAGE', inplace = True, axis = 1)
basket_sets.head()

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Build up the Frequent Items (Min. Support 0.03)
frequent_itemsets = apriori(basket_sets, min_support = 0.03, use_colnames = True)
frequent_itemsets.head()

,support,itemsets
0,0.032823,(3 HOOK HANGER MAGIC GARDEN)
1,0.041575,(3 PIECE SPACEBOY COOKIE CUTTER SET)
2,0.037199,(5 HOOK HANGER RED MAGIC TOADSTOOL)
3,0.102845,(6 RIBBONS RUSTIC CHARM)
4,0.032823,(ALARM CLOCK BAKELIKE GREEN)


The lift value greater than 1 indicates that customers who buy antecedent products have a high tendency to buy consequent products. Therefore, rules will be formed with a threshold value for the lift of 1

In [14]:
# Create the Rules
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1).sort_values(by = ['lift'], ascending = False)
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,(JUMBO BAG WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.100656,0.245077,0.037199,0.369565,1.507958,0.012531,1.197465
2,(6 RIBBONS RUSTIC CHARM),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.102845,0.245077,0.035011,0.340426,1.389058,0.009806,1.144561
3,(ROUND SNACK BOXES SET OF4 WOODLAND ),(6 RIBBONS RUSTIC CHARM),0.245077,0.102845,0.035011,0.142857,1.389058,0.009806,1.046681
113,(REGENCY CAKESTAND 3 TIER),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.137856,0.245077,0.039387,0.285714,1.165816,0.005602,1.056893
112,(ROUND SNACK BOXES SET OF4 WOODLAND ),(REGENCY CAKESTAND 3 TIER),0.245077,0.137856,0.039387,0.160714,1.165816,0.005602,1.027236


In [15]:
rules.shape

(196, 9)

It is known that there are 196 association rules that were formed

Next, association rules will be filtered based on lift and confidence values. The threshold value for lift is 6, while the threshold value for confidence is 0.6. The confidence value shows the percentage of antecedent purchases followed by consequent purchases

In [16]:
# Filter Lift (6) and Confidence (0.6)
filtered = rules[(rules['lift'] >= 6) & (rules['confidence'] >= 0.6)].sort_values(by = ['lift'], ascending = False)
filtered.head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
138,(SPACEBOY CHILDRENS CUP),(SPACEBOY CHILDRENS BOWL),0.043764,0.041575,0.037199,0.850000,20.444737,0.03538,6.389497
139,(SPACEBOY CHILDRENS BOWL),(SPACEBOY CHILDRENS CUP),0.041575,0.043764,0.037199,0.894737,20.444737,0.03538,9.084245


In [17]:
filtered.shape

(19, 9)

It is known that there are 19 filtered association rules

In Germany, the probability of purchasing "SPACEBOY CHILDRENS BOWL" (antecedents) and "SPACEBOY CHILDRENS CUP" (consequent) simultaneously is 0.037199 of the total transactions. Based on the purchase of the "SPACEBOY CHILDRENS BOWL", 89.47% of customers also purchased the "SPACEBOY CHILDRENS CUP". The value of the lift is greater than 1 so that customers who buy "SPACEBOY CHILDRENS BOWL" have a high tendency to buy "SPACEBOY CHILDRENS CUP" as well

In Germany, the probability of purchasing "SPACEBOY CHILDRENS CUP" (antecedents) and "SPACEBOY CHILDRENS BOWL" (consequent) simultaneously is 0.037199 of the total transactions. Based on the purchase of the "SPACEBOY CHILDRENS CUP", 85.00% of customers also purchased the "SPACEBOY CHILDRENS BOWL". The value of the lift is greater than 1 so that customers who buy "SPACEBOY CHILDRENS CUP" have a high tendency to buy "SPACEBOY CHILDRENS BOWL" as well